##Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [4]:
# -*- coding: utf8 -*-
import codecs
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pickle
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False

In [5]:
paradigmFile="2015-Data/MGC-150815-extend-01-X-paradigmes.csv"
goldFile="2015-Data/MGC-150812-total-paradigmes.csv"

###Préparation des cases du paradigme

In [6]:
casesPrincipales= [
        'inf', 'pi1S', 'pi2S', 'pi3S', 'pi1P', 'pi2P', 'pi3P', 'ii1S',
        'ii2S', 'ii3S', 'ii1P', 'ii2P', 'ii3P', 
        'fi1S', 'fi2S', 'fi3S', 'fi1P', 'fi2P',
        'fi3P', 'pI2S', 'pI1P', 'pI2P', 'ps1S', 'ps2S', 'ps3S', 'ps1P',
        'ps2P', 'ps3P', 
        'pc1S', 'pc2S', 'pc3S', 'pc1P', 'pc2P', 'pc3P', 'pP',
        'ppMS', 'ppMP', 'ppFS', 'ppFP'
            ]
casesSecondaires= [
       'ai1S', 'ai2S', 'ai3S', 'ai1P', 'ai2P', 'ai3P', 'is1S', 'is2S', 'is3S', 'is1P', 'is2P', 'is3P'
            ]
casesTotales=casesPrincipales+casesSecondaires
listeCases=casesTotales

###Dédoubler les lignes avec des surabondances dans *colonne*
>identifier une ligne avec surabondance

>>ajouter les lignes correspondant à chaque valeur

>>ajouter le numéro de la ligne initiale dans les lignes à supprimer

>supprimer les lignes avec surabondance

NB : il faut préparer le tableau pour avoir une indexation qui permette l'ajout des valeurs individuelles et la suppression des lignes de surabondances

In [7]:
paradigmes=pd.read_csv(paradigmFile,sep=";",encoding="utf8")
del paradigmes[u"Unnamed: 0"]
sampleCases=paradigmes.columns.values.tolist()
sampleCases.remove(u"lexeme")

In [8]:
GOLD=pd.read_csv(goldFile,sep=";",encoding="utf8")
del GOLD[u"Unnamed: 0"]
goldCases=GOLD.columns.values.tolist()
goldCases.remove(u"lexeme")

In [9]:
listeTest=paradigmes.dropna(thresh=3)["lexeme"].values.tolist()
for i,element in enumerate(listeTest):
    print i, element,

0 abaisser 1 abandonner 2 abattre 3 abonder 4 aborder 5 aboutir 6 aboyer 7 abriter 8 abréger 9 absorber 10 absoudre 11 abstenir 12 abstraire 13 abuser 14 abîmer 15 accabler 16 accaparer 17 accentuer 18 accepter 19 acclamer 20 accommoder 21 accompagner 22 accomplir 23 accorder 24 accoucher 25 accouder 26 accourir 27 accrocher 28 accroupir 29 accroître 30 accueillir 31 accumuler 32 accuser 33 accélérer 34 acharner 35 acheter 36 achever 37 acquiescer 38 acquitter 39 acquérir 40 actionner 41 activer 42 adapter 43 adhérer 44 admettre 45 admirer 46 adonner 47 adopter 48 adorer 49 adosser 50 adoucir 51 adresser 52 advenir 53 affaiblir 54 affairer 55 affaisser 56 affaler 57 affamer 58 affecter 59 affermir 60 afficher 61 affirmer 62 affliger 63 affoler 64 affranchir 65 affronter 66 agacer 67 agenouiller 68 agglutiner 69 agir 70 agiter 71 agoniser 72 agrandir 73 agresser 74 agripper 75 aider 76 aimer 77 ajouter 78 ajuster 79 alerter 80 aligner 81 alimenter 82 allaiter 83 aller 84 allier 85 allon

In [53]:
paradigmes[sampleCases].stack().value_counts(dropna=True).sum()

14980

In [54]:
paradigmes[sampleCases].dropna(thresh=2).stack().value_counts(dropna=True).sum()

14126

In [13]:
paradigmes[paradigmes["lexeme"]=="asseoir"].dropna(thresh=3)

,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ppFP,ppFS,ppMP,ppMS,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S
199,asseoir,asim,asi,NaN,asi,asir,asi,NaN,NaN,NaN,...,asiz,asiz,NaN,NaN,NaN,aswa,NaN,NaN,NaN,asEj


In [14]:
GOLD[GOLD["lexeme"]=="asseoir"].dropna(thresh=3)

,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ppFP,ppFS,ppMP,ppMS,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S
398,asseoir,asim,asi,asit,asi,asir,asi,"asjèrô,aswarô","asjèrè,aswarè","asjère,asware",...,asiz,asiz,asi,asi,"asèjô,aswajô","asEj,aswa","asèje,aswaje","asEj,aswa","asEj,aswa","asEj,aswa"


In [48]:
GOLD[goldCases].stack().value_counts(dropna=True).sum()

327258

In [49]:
GOLD[goldCases].dropna(thresh=2).stack().value_counts(dropna=True).sum()

327198

In [50]:
GOLD[goldCases].dropna(thresh=2)

,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,fi2S,...,ppFP,ppFS,ppMP,ppMS,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S
0,abEsam,abEsE,abEsat,abEsa,abEsEr,abEsa,abEs6rô,abEs6rE,abEs6re,abEs6ra,...,abEse,abEse,abEse,abEse,abEsjô,abEs,abEsje,abEs,abEs,abEs
1,abâdOnam,abâdOnE,abâdOnat,abâdOna,abâdOnEr,abâdOna,abâdOn6rô,abâdOn6rE,abâdOn6re,abâdOn6ra,...,abâdOne,abâdOne,abâdOne,abâdOne,abâdOnjô,abâdOn,abâdOnje,abâdOn,abâdOn,abâdOn
2,abazurdim,abazurdi,abazurdit,abazurdi,abazurdir,abazurdi,abazurdirô,abazurdirE,abazurdire,abazurdira,...,abazurdi,abazurdi,abazurdi,abazurdi,abazurdisjô,abazurdis,abazurdisje,abazurdis,abazurdis,abazurdis
3,abatim,abati,abatit,abati,abatir,abati,abatrô,abatrE,abatre,abatra,...,abaty,abaty,abaty,abaty,abatjô,abat,abatje,abat,abat,abat
4,absEdam,absEdE,absEdat,absEda,absEdEr,absEda,absEd6rô,absEd6rE,absEd6re,absEd6ra,...,NaN,NaN,NaN,absEde,absEdjô,absEd,absEdje,absEd,absEd,absEd
5,abdikam,abdikE,abdikat,abdika,abdikEr,abdika,abdik6rô,abdik6rE,abdik6re,abdik6ra,...,abdike,abdike,abdike,abdike,abdikjô,abdik,abdikje,abdik,abdik,abdik
6,abEkam,abEkE,abEkat,abEka,abEkEr,abEka,abEk6rô,abEk6rE,abEk6re,abEk6ra,...,abEke,abEke,abEke,abEke,abEkjô,abEk,abEkje,abEk,abEk,abEk
7,abEram,abErE,abErat,abEra,abErEr,abEra,abEr6rô,abEr6rE,abEr6re,abEr6ra,...,NaN,NaN,NaN,abEre,abErjô,abEr,abErje,abEr,abEr,abEr
8,abOram,abOrE,abOrat,abOra,abOrEr,abOra,abOr6rô,abOr6rE,abOr6re,abOr6ra,...,abOre,abOre,abOre,abOre,abOrjô,abOr,abOrje,abOr,abOr,abOr
9,abZyram,abZyrE,abZyrat,abZyra,abZyrEr,abZyra,abZyr6rô,abZyr6rE,abZyr6re,abZyr6ra,...,abZyre,abZyre,abZyre,abZyre,abZyrjô,abZyr,abZyrje,abZyr,abZyr,abZyr
